### data creation

In [1]:
def cut_data_by_qtime(qtime=0.5):
    '''le problème principal est que la liste d articles étaient presque complètes avant la première interaction
        en cutant par un quantile du click_timestamp, on diminuer le nombre de clics mais pas le nombre d'articles
        et si on cut par un quantile items.created_at_ts, on aura plus d interactions
        
        dans l ideal il faut cutter les items par un quantile items.created_at_ts
                                et les interactions par un quantile click_timestamp
                                en virant toutes les interactions avec des items cuttés
                                mais on peut renconter un grand saut dans le nombre d interactions'''

    cut_click=clics.click_timestamp.quantile(qtime)
    #cut_items=items.created_at_ts.quantile(q)
    #fusion items et items2vec "embedding"
    items_=pd.concat([items,pd.DataFrame(items2vec)], axis=1)
    #data cut
    clics__=clics.loc[clics.click_timestamp<cut_click].copy()
    items__=items_.loc[items_.created_at_ts<cut_click].copy()
    
    #merge clics & items, pour retirer les articles non connus
    data=clics__.loc[clics__.click_article_id.isin(set(items__.article_id))].copy()
    data.rename(columns={'click_article_id':'article_id'}, inplace = True)
    #display(data)
    #1 ligne par item interagi pour chaque user
    data_map=data[['user_id','article_id']].groupby(['user_id', 'article_id']).size().to_frame().reset_index()
    data_map.columns=['user_id','article_id','nb']
    #display(data_map)
    #extraction item2vec
    items2vec_=items__.iloc[:,5:].to_numpy()
    #creation des dicos inner_id/raw_id et raw_id/inner_id
    dico_inner_raw_iid = dict( (inner_iid,iid) for inner_iid,iid in enumerate(items__.article_id ))
    dico_raw_inner_iid = dict( (iid,inner_iid) for inner_iid,iid in enumerate(items__.article_id ))
    return data_map, items__, items2vec_, dico_inner_raw_iid, dico_raw_inner_iid

In [3]:
#globo user data
import pandas as pd
#article content embedding : np matrix 364047 articles / 250D
#embeddings can be used by Neural Networks to represent their content
items2vec = pd.read_pickle(r'data/news-portal-user-interactions-by-globocom/articles_embeddings.pickle')
#metadata info about 364047 articles
items = pd.read_csv('data/news-portal-user-interactions-by-globocom/articles_metadata.csv', delimiter = ',')
#all_click
for i in range(365):
    temp=pd.read_csv('data/news-portal-user-interactions-by-globocom/clicks/clicks/clicks_hour_'+"%03d" % i+'.csv', delimiter = ',')
    if i==0:
        clics=temp
    else:
        clics=pd.concat([clics, temp])
clics=clics.reset_index(drop=True)

data_map,items_df, i2vec, dic_ir, dic_ri=cut_data_by_qtime(qtime=0.01)

In [4]:
data_map

,user_id,article_id,nb
0,0,68866,1
1,0,157541,1
2,1,96663,1
3,1,235840,1
4,2,30970,1
...,...,...,...
27919,10721,156543,1
27920,10721,207122,1
27921,10722,159359,1
27922,10723,207122,1


In [3]:
#calcul des user profils
import numpy as np
from sklearn import preprocessing
def _build_users_profile(uid, click_df,emb_matrix,dic_ri):
    '''calcul du profil embedding pour un uid
    à partir des interactions du train'''
    #on recupere le dataframe d'interaction pour l'uid ciblé
    click_uid_df = click_df.loc[click_df.user_id==uid]
    #on recupere les embeddings des items vus par l'uid
    user_item_profiles = np.array([emb_matrix[dic_ri[iid]] for iid in click_uid_df['article_id']])
    #on recupere le nombre de click sur les articles par l'uid afin d apporter une ponderation à la moyenne des embedding pour le calcul du profil
    user_item_strengths = np.array(click_uid_df['nb']).reshape(-1,1) #-1 veut dire unknow dim
    #on pondere la localisation embedding de chaque item par le nombre d'interactions puis on somme afin d'obtenir le profile qui est un barycentre
    user_item_strengths_weighted_avg = np.sum(np.multiply(user_item_profiles,user_item_strengths), axis=0) / np.sum(user_item_strengths)
    user_profile_norm = preprocessing.normalize(np.reshape(user_item_strengths_weighted_avg,(1,250)))
    return user_profile_norm

def _build_users_profiles(click_df, emb_matrix,dic_ri): 
    '''calcul des profils de tous les uid sous forme de dic {uid:profil}
    à partir des interactions du train'''
    user_profiles = {}
    for uid in click_df.user_id.unique():
        user_profiles[uid] = _build_users_profile(uid, click_df, emb_matrix, dic_ri)
    return user_profiles

profils=_build_users_profiles(data_map, i2vec,dic_ri)

In [4]:
from surprise import Reader, Dataset,SVDpp, dump

#fit du collab filtering
'fit du modele SVDpp'
reader = Reader(rating_scale=(0,5))
#lecture du train par surprise
data = Dataset.load_from_df(data_map[['user_id','article_id','nb']], reader)
#construction du trainset surprise 
trainset = data.build_full_trainset()
algo = SVDpp(n_epochs=20, lr_all=0.007, reg_all=0.1)
algo.fit(trainset)

In [5]:
# Dump algorithm and reload it.
import os
local_path = "./2AzBlob"
#os.mkdir(local_path)

In [6]:
#tout en un seul pickle pour essayer de regler les problemes server 500 en prod de l azure function
import pickle

local_file='all.pkl'
file_path = os.path.join(local_path, local_file)
file = open(file_path,'wb')

pickle.dump(algo, file)
pickle.dump(data_map, file)
pickle.dump(items_df, file)
pickle.dump(i2vec, file)
pickle.dump(dic_ir, file)
pickle.dump(dic_ri, file)
pickle.dump(profils, file)

file.close()

In [ ]:
import pickle
local_file='algo.pkl'
file_path = os.path.join(local_path, local_file)
pickle.dump(file_path, algo=algo)
#on binarise/serialise le numpy array du word embedding
local_file='data_map.pkl'
file_path = os.path.join(local_path, local_file)
with open(file_path, 'wb') as f:
    pickle.dump(data_map, f)
    
local_file='items_df.pkl'
file_path = os.path.join(local_path, local_file)
with open(file_path, 'wb') as f:
    pickle.dump(items_df, f)
    
local_file='i2vec.pkl'
file_path = os.path.join(local_path, local_file)
with open(file_path, 'wb') as f:
    pickle.dump(i2vec, f)

local_file='dic_ir.pkl'
file_path = os.path.join(local_path, local_file)
with open(file_path, 'wb') as f:
    pickle.dump(dic_ir, f)

local_file='dic_ri.pkl'
file_path = os.path.join(local_path, local_file)
with open(file_path, 'wb') as f:
    pickle.dump(dic_ri, f)

local_file='profils.pkl'
file_path = os.path.join(local_path, local_file)
with open(file_path, 'wb') as f:
    pickle.dump(profils, f)

In [10]:
import os, uuid
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__

try:
    print("Azure Blob Storage v" + __version__ + " - Python quickstart sample")
        # recuperation de la chaine de connexion au azure blob storage à partir de la variabale environmentale
    connect_str = os.getenv('AZURE_STORAGE_CONNECTION_STRING')

        # Creation du service_client blob qui permettra de creer un conteneur
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
        # Creation d'un nom unique aleatoire grace a la fonction uuid4 pour le conteneur blob
    container_name = "522f43be-48af-4f3d-a2d7-4deba1749d31" #str(uuid.uuid4())
        # Creation du conteneur si il n'existe pas
    #container_client = blob_service_client.create_container(container_name)
    container_client = blob_service_client.get_container_client(container_name)


    toblob_list=['all.pkl']
    #toblob_list=['data_map.pkl','items_df.pkl','i2vec.pkl','dic_ir.pkl','dic_ri.pkl','algo.pkl','profils.pkl']

    # Create a blob client using the local file name as the name for the blob
    for i in toblob_list:
        local_file = i
        upload_file_path = os.path.join(local_path, local_file)
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=local_file)

        print("\nUploading to Azure Storage as blob:\n\t" + local_file)

        # Upload the created file
        with open(upload_file_path, "rb") as data:
            blob_client.upload_blob(data)


except Exception as ex:
    print('Exception:')
    print(ex)

Azure Blob Storage v12.12.0 - Python quickstart sample

Uploading to Azure Storage as blob:
	all.pkl


In [13]:
print("\nListing blobs...")
#container_client = blob_service_client.get_container_client(container_name)
# List the blobs in the container
blob_list = container_client.list_blobs()
for blob in blob_list:
    print("\t" + blob.name)


Listing blobs...
	algo.pkl
	all.pkl
	data_map.pkl
	dic_ir.pkl
	dic_ri.pkl
	i2vec.pkl
	items_df.pkl
	profils.pkl


In [30]:
from io import BytesIO


In [34]:
blob_client = blob_service_client.get_blob_client(container=container_name, blob='all.pkl')
hh=(blob_client.download_blob(0).readall())

In [36]:
from io import BytesIO
blob_to_read = BytesIO(hh)
#with open(hh, 'rb') as file:
with blob_to_read as file:
#blob_client = blob_service_client.get_blob_client(container=container_name, blob='all.pkl')
#file = open(blob_client.download_blob(0).readall(), 'rb')
    algo_ = pickle.load(file)
    data_map_ = pickle.load(file)
    items_df_ = pickle.load(file)
    i2vec_ = pickle.load(file)
    dic_ir_= pickle.load(file)
    dic_ri_ = pickle.load(file)
    profils_= pickle.load(file)
#file.close()

In [38]:
data_map_

,user_id,article_id,nb
0,0,68866,1
1,0,157541,1
2,1,96663,1
3,1,235840,1
4,2,30970,1
...,...,...,...
27919,10721,156543,1
27920,10721,207122,1
27921,10722,159359,1
27922,10723,207122,1


In [19]:
%%time
local_file='all.pkl'
file_path = os.path.join(local_path, local_file)
file = open(file_path,'rb')

#blob_client = blob_service_client.get_blob_client(container=container_name, blob='all.pkl')
#file = open(blob_client.download_blob(0).readall(), 'rb')
algo_ = pickle.load(file)
data_map_ = pickle.load(file)
items_df_ = pickle.load(file)
i2vec_ = pickle.load(file)
dic_ir_= pickle.load(file)
dic_ri_ = pickle.load(file)
profils_= pickle.load(file)
file.close()

CPU times: total: 219 ms
Wall time: 222 ms


In [47]:
%%time
#chargement du blob
#blob_client = BlobClient.from_connection_string(connection_string, container_name, blob_name)
blob_client = blob_service_client.get_blob_client(container=container_name, blob='data_map.pkl')
data_map_fb=pickle.loads(blob_client.download_blob(0).readall())
blob_client = blob_service_client.get_blob_client(container=container_name, blob='items_df.pkl')
items_df_fb=pickle.loads(blob_client.download_blob(0).readall())
blob_client = blob_service_client.get_blob_client(container=container_name, blob='i2vec.pkl')
i2vec_fb=pickle.loads(blob_client.download_blob(0).readall())
blob_client = blob_service_client.get_blob_client(container=container_name, blob='dic_ir.pkl')
dic_ir_fb=pickle.loads(blob_client.download_blob(0).readall())
blob_client = blob_service_client.get_blob_client(container=container_name, blob='dic_ri.pkl')
dic_ri_fb=pickle.loads(blob_client.download_blob(0).readall())
blob_client = blob_service_client.get_blob_client(container=container_name, blob='profils.pkl')
profils_fb=pickle.loads(blob_client.download_blob(0).readall())
blob_client = blob_service_client.get_blob_client(container=container_name, blob='algo.pkl')
algo_fb=pickle.loads(blob_client.download_blob(0).readall())

CPU times: total: 5.61 s
Wall time: 22.1 s


In [44]:
container_name

'522f43be-48af-4f3d-a2d7-4deba1749d31'

### reco

In [48]:
import numpy as np
from sklearn import preprocessing
import scipy
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Reader, Dataset,SVDpp

class ContentBasedRecommender:
    '''le fit calcule les users profiles'''
    
    MODEL_NAME = 'Content-Based'
    

    def __init__(self,data_map,i2vec,dic_ri,dic_ir,profils):
        '''constructeur'''
        #dico inner raw iid
        self.dic_ir = dic_ir
        #dico raw inner iid
        self.dic_ri = dic_ri
        self.items_embedding = i2vec
        #les train interactions servent à:
            #calculer les profils
            #ignorer les interactions dans la reco
        self.train_user_interact = data_map
        self.user_profiles = profils
    
    
    def get_model_name(self):
        '''renvoie le nom du modèle'''
        return self.MODEL_NAME
    

    def _get_similar_items_to_user_profile(self, uid):#, topn=100000):
        '''renvoie une liste d items similaires au profil de l'uid
            liste de couples (raw iid, cosine) triés par cosine decroissant'''
        #on calcul distances profil uid vs articles : sous forme 2D 1 ligne / nombre iid colonnes dans l'ordre inner du wemb
        cosine_similarities = cosine_similarity(self.user_profiles[uid], self.items_embedding)
        #on tri par distance: argsort donne les indices par scores croissants /flatten pour passer en 1D
        #on recupère un array des inner_iid de taille topn : les plus proches du profil sont à droite
        similar_indices = cosine_similarities.argsort().flatten()#[-topn:]
                                                        ################################################################################
        #on tri : sorted(iterable, key=None, reverse=False)
        # l'iterable est une liste de couples (raw_iid, cosine similarites)
        #on tri par similarité décroissante
        similar_items = [(self.dic_ir[i], cosine_similarities[0,i]) for i in similar_indices[::-1]]
        return similar_items
    
    #methode qui recommande en ignorant les articles deja vus
    def recommend_items(self, uid, topn=5):
        '''renvoie une liste de reco similaires au profil de l uid
            triés par cosine decroissant
             sans les interactions du train
                sous forme de dataframe'''
        #on recupere la liste brute de 1200 reco classés par cosine decroissant
        similar_items = self._get_similar_items_to_user_profile(uid)
        #on recupere la liste des iid a ignorer car deja vus
        #items_to_ignore = self.train_user_interact.loc[self.train_user_interact.user_id==uid,"article_id"].tolist()
        items_to_ignore = set(self.train_user_interact.loc[self.train_user_interact.user_id==uid].article_id)
        #on enleve les deja vus
        #similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        recommendations_df = pd.DataFrame(similar_items, columns=['article_id', 'cb_cosine_with_profile'])
        reco = recommendations_df.loc[recommendations_df.article_id.isin(items_to_ignore)==False].copy()
        if topn>0:
            reco=reco.head(topn)
        return reco


class CollabFiltRecommender:
    
    MODEL_NAME = 'Collaborative-Filtering-SVDpp'
    
    #constructeur
    def __init__(self,algo,data_map):
        '''constructeur'''
        #les train interactions servent à:
            #calculer les profils
            #ignorer les interactions dans la reco
        #self.train_user_interact = data_map
        self.algo=algo
        self.train_user_interact = data_map
        
    #methode qui renvoie le nom du modele    
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self,uid,topn=5):
        '''renvoie une liste de prediction pour un uid
            le SVD ne peut sortir des prédictions que pour les uid et iid dispo dans le train'''
        #prediction fonctionne avec les raws id
        #https://towardsdatascience.com/difference-between-apply-and-transform-in-pandas-242e5cf32705
        iid_to_ignore=set(self.train_user_interact.loc[self.train_user_interact.user_id==uid].article_id)
        items2pred=pd.DataFrame(set(self.train_user_interact.article_id)-iid_to_ignore,columns=['article_id'])
        items2pred['pred']=items2pred['article_id'].apply(lambda x:self.algo.predict(uid=uid, iid=x)[3])
        #pred['detail']=pred['article_id'].apply(lambda x:cf.predict(uid=uid, iid=x)[4])
        if topn==0:
            recommendations_df=items2pred.loc[:,['article_id','pred']].sort_values(by='pred', ascending=False)
        else:
            recommendations_df=items2pred.loc[:,['article_id','pred']].sort_values(by='pred', ascending=False).head(topn)

        return recommendations_df

class PopularityFiltRecommender:
    
    MODEL_NAME = 'Popularity-Filtering'
    
    #constructeur
    def __init__(self,data_map):
        '''constructeur'''
        self.train_user_interact = data_map
        
    #methode qui renvoie le nom du modele    
    def get_model_name(self):
        return self.MODEL_NAME
    
    #fit du modèle
    def fit(self):
        self.raw_reco=self.train_user_interact[['nb','article_id']].groupby(['article_id']).sum().sort_values(by=['nb'],ascending=False).reset_index()

    def recommend_items(self,uid,topn=5):
        '''renvoie une liste de prediction pour un uid'''
        if topn==0:
            recommendations_df=self.raw_reco
        else:
            recommendations_df=self.raw_reco.head(topn)
        return recommendations_df

class HybridRecommender:
    
    MODEL_NAME = 'Hybrid-Filtering'
    
    #constructeur
    def __init__(self,data_map,i2vec,dic_ri,dic_ir, profils, algo):
        '''constructeur'''
        self.train_user_interact = data_map
        self.dic_ir = dic_ir
        self.dic_ri = dic_ri
        self.items_embedding = i2vec
        self.user_profiles = profils
        self.algo=algo
    
    def get_model_name(self):
        return self.MODEL_NAME
    
    #fit du modèle
    def fit(self):
        '''le fit consiste a fitter les sous modèles'''
        self.cf_model = CollabFiltRecommender(self.algo,self.train_user_interact)
        self.cb_model = ContentBasedRecommender(self.train_user_interact,self.items_embedding,self.dic_ri,self.dic_ir,self.user_profiles)
        self.pf_model = PopularityFiltRecommender(self.train_user_interact)
        self.pf_model.fit()
        
    def recommend_items(self,uid,topn=5):
        '''renvoie une liste de prediction pour un uid'''
        #si uid pas connu du train: seul le popularity recommander fonctionne dans notre cas
        #on pourrait également concevoir un social reco basé sur un social profil, encore faudrait il avoir assez d 'infos'
        if uid not in set(self.train_user_interact.user_id):
            #parfait quand l'uid n'est pas connu du train, n a pas  d'interactions
            reco=self.pf_model.recommend_items(uid,0)
        else:
                # content based : permet de donner des iid qui n'ont pas d'interactions mais qui étaient disponibles
            reco_cb=self.cb_model.recommend_items(uid,0)
                # pour la normalisation, le but est de garder les meilleurs, donc les outliers positifs
                #il vaut mieux dans ce cas diviser par le max que par une standard deviation 
                #qui ne rendra pas forcement les outliers similaires entre facteurs pour une pondéaration
            #reco_cb['norm_cb']=(reco_cb.cb_cosine_with_profile-reco_cb.cb_cosine_with_profile.median())/(reco_cb.cb_cosine_with_profile.max()-reco_cb.cb_cosine_with_profile.median())
            reco_cb['rank_cb']=reco_cb.cb_cosine_with_profile.rank(ascending=False)
                # donne de bons resultats mais necessite des uid et des iid qui ont des interactions dans le train
            reco_cf=self.cf_model.recommend_items(uid,0)
            #reco_cf['norm_cf']=(reco_cf.pred-reco_cf.pred.median())/(reco_cf.pred.max()-reco_cf.pred.median())
            reco_cf['rank_cf']=reco_cf.pred.rank(ascending=False)
                #comment les pondérer? 80/20 normalisé par exemple?
                #en gros la pondération est anecdotique: on privilegie CF et quand on a pas de CF, on met la note de CB
            reco = reco_cb.merge(reco_cf,how='outer', on='article_id')
                #on fill les norm_cf vides (iid inconnu du train) par norm_cv avant pondération
                #on pourait également filler les iid peu interagis afin de favoriser le content dans ce cas la
            reco.loc[reco['rank_cf'].isnull(),'rank_cf'] = reco['rank_cb']
            reco['rank']=0.99999*reco['rank_cf']+0.00001*reco['rank_cb']
            
            reco=reco.sort_values(by='rank', ascending=True)


        if topn==0:
            recommendations_df=reco
        else:
            recommendations_df=reco.head(topn)
            
        return recommendations_df.reset_index(drop=True)
        #return recommendations_df

In [58]:
hr_model = HybridRecommender(data_map_fb,i2vec_fb,dic_ri_fb,dic_ir_fb,profils_fb,algo_fb)
hr_model.fit()

In [61]:
%%time
hr_model.recommend_items(7).article_id.to_list()

CPU times: total: 875 ms
Wall time: 785 ms


[285781, 97948, 207742, 353724, 232088]

In [69]:
data_map_fb.loc[(data_map_fb.user_id==2) & (data_map_fb.article_id==30970)].nb

4    1
Name: nb, dtype: int64

In [1]:
import pickle 
print(pickle.format_version)


4.0
